In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Load MovieLens dataset (replace with your file paths)
movies = pd.read_csv(r'.\movies.csv')
ratings = pd.read_csv(r'.\ratings.csv')

# Merge movies and ratings
movie_ratings = pd.merge(ratings, movies, on='movieId')

# Create a user-item matrix
user_movie_ratings = movie_ratings.pivot_table(index='userId', columns='title', values='rating')

# Fill NaN values with 0 (user hasn't rated the movie)
user_movie_ratings = user_movie_ratings.fillna(0)

# Transpose the user-item matrix for item-based collaborative filtering
item_similarity = cosine_similarity(user_movie_ratings.T)

# Create a similarity dataframe
item_similarity_df = pd.DataFrame(item_similarity, index=user_movie_ratings.columns, columns=user_movie_ratings.columns)

# Recommend movies based on user input
def recommend_movies():
    print("Enter your movie preferences:")
    user_input = {}
    
    while True:
        movie = input("Enter a movie title (or type 'done' to finish): ")
        if movie.lower() == 'done':
            break

        # Perform a partial case-insensitive match for the movie title
        matched_titles = [title for title in user_movie_ratings.columns if movie.lower() in title.lower()]
        
        if not matched_titles:
            print(f"No matches found for '{movie}' in the dataset. Please enter a different title.")
            continue

        print("Matched Titles:")
        for i, matched_title in enumerate(matched_titles, 1):
            print(f"{i}. {matched_title}")

        selection = input("Select a title by entering its number (or type 'cancel' to enter a different title): ")
        
        if selection.lower() == 'cancel':
            continue

        try:
            selected_index = int(selection) - 1
            matched_title = matched_titles[selected_index]
            rating = float(input(f"Rate '{matched_title}': "))
            user_input[matched_title] = rating
        except (ValueError, IndexError):
            print("Invalid selection. Please enter a valid number or type 'cancel' to enter a different title.")

    user_ratings = pd.Series(user_input)
    recommendations = pd.Series()

    for movie, rating in user_ratings.items():
        similar_scores = item_similarity_df[movie] * rating
        similar_movies = similar_scores.sort_values(ascending=False)

        # Exclude movies that the user has already inputted
        similar_movies = similar_movies[~similar_movies.index.isin(user_ratings.index)]
        
        recommendations = pd.concat([recommendations, similar_movies], axis=1)

    recommendations = recommendations.sum(axis=1)
    recommendations = recommendations.sort_values(ascending=False)

    return recommendations

# Example usage with user input
if __name__ == "__main__":
    user_recommendations = recommend_movies()

    print("Recommended Movies:")
    print(user_recommendations.head(10))


C:\Users\LENOVO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


Enter your movie preferences:
No matches found for 'Bravheart' in the dataset. Please enter a different title.
Matched Titles:
1. Braveheart (1995)
Matched Titles:
1. Dream for an Insomniac (1996)
2. Insomnia (1997)
3. Insomnia (2002)
Matched Titles:
1. 21 Jump Street (2012)
2. 22 Jump Street (2014)
Matched Titles:
1. Ace Ventura: Pet Detective (1994)
2. Ace Ventura: When Nature Calls (1995)
Recommended Movies:
Jurassic Park (1993)                      6.690467
Forrest Gump (1994)                       6.640864
Terminator 2: Judgment Day (1991)         6.448022
Pulp Fiction (1994)                       6.336636
Mask, The (1994)                          6.306836
Seven (a.k.a. Se7en) (1995)               6.199188
Dumb & Dumber (Dumb and Dumber) (1994)    6.097959
True Lies (1994)                          6.093285
Fugitive, The (1993)                      6.080626
Batman (1989)                               6.0672
dtype: object
